# Parsing Evaluation with Chroma

In [13]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from chunking_evaluation import GeneralEvaluation
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

from lib.evaluation.segmentation.chroma.chroma_adapter import ChromaChunker
from lib.segmentation.methods.implementations.fixed_sized import FixedSizeChunker
from lib.segmentation.methods.implementations.recursive import RecursiveChunker
from lib.segmentation.methods.implementations.hierarchical import HierarchicalChunker
from lib.segmentation.methods.implementations.semantic import SemanticChunker
from lib.parsing.methods.parsers import Parsers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
chunkers = [
    FixedSizeChunker(max_tokens=128, overlap=0),
    FixedSizeChunker(max_tokens=128, overlap=26),
    FixedSizeChunker(max_tokens=256, overlap=0),
    FixedSizeChunker(max_tokens=256, overlap=52),
    FixedSizeChunker(max_tokens=512, overlap=0),
    FixedSizeChunker(max_tokens=512, overlap=104),
    FixedSizeChunker(max_tokens=1024, overlap=0),
    FixedSizeChunker(max_tokens=1024, overlap=208),
    RecursiveChunker(max_tokens=128, overlap=0),
    RecursiveChunker(max_tokens=128, overlap=26),
    RecursiveChunker(max_tokens=256, overlap=0),
    RecursiveChunker(max_tokens=256, overlap=52),
    RecursiveChunker(max_tokens=512, overlap=0),
    RecursiveChunker(max_tokens=512, overlap=104),
    RecursiveChunker(max_tokens=1024, overlap=0),
    RecursiveChunker(max_tokens=1024, overlap=208),
    SemanticChunker(max_tokens=128, similarity_threshold=90, min_tokens=64),
    SemanticChunker(max_tokens=128, similarity_threshold=70, min_tokens=64),
    SemanticChunker(max_tokens=256, similarity_threshold=90, min_tokens=128),
    SemanticChunker(max_tokens=256, similarity_threshold=70, min_tokens=128),
    SemanticChunker(max_tokens=512, similarity_threshold=90, min_tokens=256),
    SemanticChunker(max_tokens=512, similarity_threshold=70, min_tokens=256),
    SemanticChunker(max_tokens=1024, similarity_threshold=90, min_tokens=512),
    SemanticChunker(max_tokens=1024, similarity_threshold=70, min_tokens=512),
    HierarchicalChunker(max_tokens=128, max_parent_token_ratio=0.2),
    HierarchicalChunker(max_tokens=256, max_parent_token_ratio=0.2),
    HierarchicalChunker(max_tokens=512, max_parent_token_ratio=0.2),
    HierarchicalChunker(max_tokens=1024, max_parent_token_ratio=0.2),
]

In [15]:
chunkers = [
    ChromaChunker(c) for c in chunkers
]

In [16]:
# Make sure "OPENAI_API_KEY" is set in a .env file
load_dotenv()
embedding_function = OpenAIEmbeddingFunction(model_name="text-embedding-3-small")

## General Evaluation (predefined data sets)

In [5]:
evaluation = GeneralEvaluation()

results = []
pubmed_results = []

for chunker in chunkers:
    c_info = chunker.get_info()
    print(c_info)

    c_res = evaluation.run(chunker, embedding_function)
    pubmed_scores = c_res["corpora_scores"]["pubmed"]
    del c_res["corpora_scores"]

    # Add method info to all-corpus-mean scores
    c_res.update(c_info)
    results.append(c_res)

    # Get data for pubmed only
    c_row = {}
    for metric, scores in pubmed_scores.items():
        metric_name = metric.replace("_scores", "")
        c_row[f"{metric_name}_mean"] = np.mean(scores)
        c_row[f"{metric_name}_std"] = np.std(scores)

    c_row.update(c_info)
    pubmed_results.append(c_row)

2026-02-15 15:46:47,854 - INFO : Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Token indices sequence length is longer than the specified maximum sequence length for this model (25022 > 512). Running this sequence through the model will result in indexing errors


{'Method': 'Fixed-Size', 'Param': '$O=0$', 'N': 128}


2026-02-15 15:46:50,535 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:52,371 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:53,493 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:54,426 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:55,250 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:56,053 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:57,283 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:58,306 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:46:59,275 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:00,213 - INFO : HTTP

{'Method': 'Fixed-Size', 'Param': '$O=26$', 'N': 128}


2026-02-15 15:47:03,225 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:05,789 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:08,203 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:09,796 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:11,313 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:12,897 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:13,943 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:14,972 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:16,329 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 15:47:17,353 - INFO : HTTP

...

2026-02-15 16:01:22,059 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:23,264 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:24,520 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:25,949 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:26,974 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:28,304 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:29,431 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:30,350 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Hierarchical', 'Param': '$B_h=204$', 'N': 512}


2026-02-15 16:01:33,464 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:35,266 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:36,603 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:37,828 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Hierarchical', 'Param': '$B_h=409$', 'N': 1024}


2026-02-15 16:01:41,825 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-15 16:01:44,176 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [40]:
from lib.utils.chroma_export import export_results

df = pd.DataFrame(results)
combined_df = export_results(df, "chroma_results_general")

2026-02-15 17:31:04,219 - INFO : Saved table content to: /Users/matteo/Uni/7.Semester/thesis/visual-chunking/thesis/figures/tables/chroma_results_general.tex


In [42]:
pubmed_df = pd.DataFrame(pubmed_results)
combined_pubmed_df = export_results(pubmed_df, "chroma_results_pubmed")

## Synthetic Evaluation

In [17]:
from lib.evaluation.segmentation.chroma.chroma_setup import setup_evaluation_from_medical_qas, \
    get_db_path

parser = Parsers.DOCLING

db_path = get_db_path(parser)
synth_eval = setup_evaluation_from_medical_qas(
    parser,
    "awmf",
    parse_exist_ok=True,
    question_exist_ok=True
)

2026-02-16 03:08:51,535 - INFO : Chroma document corpus exists at /Users/matteo/Uni/7.Semester/thesis/visual-chunking/data/configs/chroma/docling/awmf.txt.Skipping generation.
2026-02-16 03:08:51,607 - INFO : Finished creating evaluation on the medical QA pairs.


In [35]:
results = []

for chunker in chunkers:
    c_info = chunker.get_info()
    print(c_info)

    c_res = synth_eval.run(
        chunker,
        embedding_function,
        retrieve=-1,
        db_to_save_chunks=db_path
    )
    del c_res["corpora_scores"]

    c_res.update(c_info)
    results.append(c_res)

{'Method': 'Fixed-Size', 'Param': '$O=0$', 'N': 128}


2026-02-16 15:22:13,150 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Fixed-Size', 'Param': '$O=26$', 'N': 128}


2026-02-16 15:22:17,144 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Fixed-Size', 'Param': '$O=0$', 'N': 256}


...

2026-02-16 15:23:34,770 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Hierarchical', 'Param': '$B_h=102$', 'N': 512}


2026-02-16 15:23:38,388 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Method': 'Hierarchical', 'Param': '$B_h=204$', 'N': 1024}


2026-02-16 15:23:40,329 - INFO : HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [36]:
pd.DataFrame(results)

,iou_mean,iou_std,recall_mean,recall_std,precision_omega_mean,precision_omega_std,precision_mean,precision_std,Method,Param,N
0,0.113643,0.133150,0.238281,0.288437,0.422392,0.111360,0.195618,0.205088,Fixed-Size,$O=0$,128
1,0.101444,0.107400,0.247866,0.287191,0.380840,0.115146,0.171430,0.158985,Fixed-Size,$O=26$,128
2,0.094259,0.103778,0.297087,0.322984,0.299501,0.115872,0.135110,0.139152,Fixed-Size,$O=0$,256
3,0.107986,0.100708,0.355571,0.338757,0.265148,0.099183,0.150604,0.129877,Fixed-Size,$O=52$,256
4,0.086644,0.082810,0.413899,0.351018,0.193270,0.081948,0.105359,0.094076,Fixed-Size,$O=0$,512
5,0.088050,0.084209,0.442326,0.369870,0.165769,0.071941,0.106956,0.094375,Fixed-Size,$O=104$,512
6,0.065677,0.064047,0.486344,0.375469,0.125596,0.062408,0.073598,0.067814,Fixed-Size,$O=0$,1024
7,0.067395,0.060655,0.458185,0.360091,0.109023,0.052292,0.084942,0.097188,Fixed-Size,$O=208$,1024
8,0.123441,0.142910,0.236098,0.298804,0.591056,0.145471,0.226727,0.209323,Recursive,$O=0$,128
9,0.092823,0.135251,0.170202,0.258320,0.445364,0.124418,0.169470,0.204876,Recursive,$O=26$,128


In [38]:
from lib.utils.chroma_export import export_results

df = pd.DataFrame(results)
combined_df = export_results(df, "chroma_results_concology")

2026-02-16 15:29:08,360 - INFO : Saved table content to: /Users/matteo/Uni/7.Semester/thesis/visual-chunking/thesis/figures/tables/chroma_results_concology_top_4.tex


In [ ]:
combined_df